1.导入相关包

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

2.导入数据集：用户消费分析

In [3]:
fpath=r"D:\pandas\order2021.xlsx"
df=pd.read_excel(fpath)
df.head(2)

,订单顺序编号,订单号,用户名,商品编号,订单金额,付款金额,渠道编号,平台类型,下单时间,付款时间,是否退款
0,8,sys-2021-306447069,user-104863,PR000499,499.41,480.42,渠道1,微信公众号,2021-01-01 01:05:50,2021-01-01 01:06:17,否
1,11,sys-2021-417411381,user-181957,PR000483,279.53,279.53,渠道1,APP,2021-01-01 01:36:17,2021-01-01 01:36:56,否


3.数据的初步探索

In [4]:
#查看数据的形状
df.shape

(104558, 11)

In [5]:
df.info()
#可以简单的看出来这个数据集，有些是有缺失值的

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104558 entries, 0 to 104557
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   订单顺序编号  104558 non-null  int64         
 1   订单号     104558 non-null  object        
 2   用户名     104558 non-null  object        
 3   商品编号    104558 non-null  object        
 4   订单金额    104558 non-null  float64       
 5   付款金额    104558 non-null  float64       
 6   渠道编号    104550 non-null  object        
 7   平台类型    104558 non-null  object        
 8   下单时间    104558 non-null  datetime64[ns]
 9   付款时间    104558 non-null  datetime64[ns]
 10  是否退款    104558 non-null  object        
dtypes: datetime64[ns](2), float64(2), int64(1), object(6)
memory usage: 8.8+ MB


In [6]:
#快速查看数据的描述性统计信息
df.describe()

,订单顺序编号,订单金额,付款金额,下单时间,付款时间
count,104558.000000,104558.000000,104558.000000,104558,104558
mean,52278.501415,1049.673450,1167.502610,2021-07-20 09:53:45.391122432,2021-07-21 07:13:01.848772864
min,1.000000,6.100000,-12.470000,2020-01-01 00:09:32,2021-01-01 00:09:47
25%,26139.250000,432.032500,383.662500,2021-05-07 10:24:54.500000,2021-05-07 10:11:03.750000128
50%,52278.500000,679.305000,641.235000,2021-07-25 19:36:02.500000,2021-07-25 19:10:52.500000
75%,78417.750000,1248.272500,1252.645000,2021-10-17 13:13:04,2021-10-17 12:51:52
max,104557.000000,28465.250000,83270.053829,2021-12-31 23:48:34,2021-12-31 23:49:04
std,30183.436612,1054.408156,2174.016150,NaN,NaN


4.简单数据处理

In [7]:
df.columns  #可以看到‘用户名’这一列，后面是多了一个空格的

Index(['订单顺序编号', '订单号', '用户名 ', '商品编号', '订单金额', '付款金额', '渠道编号', '平台类型', '下单时间',
       '付款时间', '是否退款'],
      dtype='object')

In [8]:
df["用户名 "].head(2)

0    user-104863
1    user-181957
Name: 用户名 , dtype: object

In [9]:
#将列名里所有的前后空格都去掉
col = df.columns.values
df.columns=[x.strip() for x in col]
df.columns

Index(['订单顺序编号', '订单号', '用户名', '商品编号', '订单金额', '付款金额', '渠道编号', '平台类型', '下单时间',
       '付款时间', '是否退款'],
      dtype='object')

5.重复值的处理

In [10]:
#检查重复值
df.duplicated().sum()


1

In [11]:
df[df.duplicated()]

,订单顺序编号,订单号,用户名,商品编号,订单金额,付款金额,渠道编号,平台类型,下单时间,付款时间,是否退款
104557,148,sys-2021-355060894,user-214122,PR000826,205.78,2044.27572,渠道1,APP,2021-01-01 14:44:49,2021-01-01 14:45:21,是


In [12]:
#删除重复值
df.drop_duplicates(inplace=True)
df.shape

(104557, 11)

In [13]:
#索引重置
df.index=range(df.shape[0])
df.index

RangeIndex(start=0, stop=104557, step=1)

6.异常值处理

In [14]:
df.describe()

,订单顺序编号,订单金额,付款金额,下单时间,付款时间
count,104557.000000,104557.000000,104557.000000,104557,104557
mean,52279.000000,1049.681521,1167.494225,2021-07-20 09:56:30.492803072,2021-07-21 07:15:47.684258048
min,1.000000,6.100000,-12.470000,2020-01-01 00:09:32,2021-01-01 00:09:47
25%,26140.000000,432.040000,383.660000,2021-05-07 10:25:41,2021-05-07 10:16:51
50%,52279.000000,679.320000,641.230000,2021-07-25 19:36:45,2021-07-25 19:11:40
75%,78418.000000,1248.280000,1252.630000,2021-10-17 13:14:10,2021-10-17 12:52:26
max,104557.000000,28465.250000,83270.053829,2021-12-31 23:48:34,2021-12-31 23:49:04
std,30183.150385,1054.409968,2174.024855,NaN,NaN


In [15]:
#利用Z-Score绝对值>3，去除掉订单金额异常值
df['Z_Score']=np.abs(stats.zscore(df['订单金额']))
df[df['Z_Score']>3]
df[df['Z_Score']>3].shape
#删掉异常值行，以及辅助列
df = df[df['Z_Score']<3].drop(columns=['Z_Score'])
df.shape

(102486, 11)

In [16]:
#基于业务的逻辑判断，付款金额应该是>0的
df=df[df["付款金额"]>0]
#并且付款金额应该不超过订单金额的
df.订单金额>=df.付款金额

0          True
1          True
2          True
3          True
4          True
          ...  
104552     True
104553     True
104554    False
104555     True
104556     True
Length: 102305, dtype: bool

In [17]:
#里面有False，则需要处理
df=df[df.订单金额>=df.付款金额]
df.shape

(100319, 11)

In [18]:
#按照逻辑，订单号一般只出现一次，是唯一的
# 找出重复的订单号
duplicates = df[df['订单号'].duplicated(keep=False)]
# 提取重复订单号对应的用户名
duplicates[['订单号', '用户名','渠道编号']].sort_values(by='订单号').head(10)

,订单号,用户名,渠道编号
77652,sys-2021-212049970,user-175996,渠道13
80613,sys-2021-212049970,user-120182,渠道13
17330,sys-2021-237885771,user-252305,渠道4
89181,sys-2021-237885771,user-122268,渠道14
92275,sys-2021-255321366,user-256511,渠道15
35250,sys-2021-255321366,user-137973,渠道7
9530,sys-2021-259110469,user-107004,渠道2
41233,sys-2021-259110469,user-152217,渠道9
81564,sys-2021-280013984,user-299453,渠道13
22070,sys-2021-280013984,user-159781,渠道5


In [19]:
#发现重复的订单号对应着不同的用户名，但是甚至有的来源于同一个渠道，这些是异常值
#可能是订单号输入错误了，可以将重复出现的订单号在末尾加上-1-2...作为标记
if not duplicates.empty:
    for order_id in duplicates["订单号"].unique():
        #找到当前重复的行
        idx=df[df["订单号"] == order_id].index
        #为每个重复行添加序号
        df.loc[idx,'订单号'] = [f"{order_id}-{i+1}" for i in range(len(idx))]


In [20]:
#对索引重新编号
df.index=range(df.shape[0])

7.缺失值处理

In [21]:
#查看缺失值的个数
df.isnull().sum()

订单顺序编号    0
订单号       0
用户名       0
商品编号      0
订单金额      0
付款金额      0
渠道编号      7
平台类型      0
下单时间      0
付款时间      0
是否退款      0
dtype: int64

In [22]:
#只有渠道编号有7个缺失值
df[df["渠道编号"].isnull()]

,订单顺序编号,订单号,用户名,商品编号,订单金额,付款金额,渠道编号,平台类型,下单时间,付款时间,是否退款
100312,11598,sys-2021-353765060,user-120690,PR000297,534.12,477.10,NaN,web网站,2021-03-02 10:11:38,2021-03-02 10:11:55,否
100313,11639,sys-2021-339868263,user-264491,PR000714,206.33,206.33,NaN,微信公众号,2021-03-02 14:02:58,2021-03-02 14:03:22,否
100314,14234,sys-2021-313502796,user-180054,PR000423,669.09,669.09,NaN,微信公众号,2021-03-16 15:13:31,2021-03-16 15:13:55,否
100315,35716,sys-2021-300339928,user-270141,PR000984,328.83,295.47,NaN,微信公众号,2021-06-06 11:03:46,2021-06-06 11:04:19,否
100316,55444,sys-2021-286784634,user-183770,PR000987,488.07,476.81,NaN,APP,2021-08-04 18:53:34,2021-08-04 18:53:49,否
100317,77890,sys-2021-251942165,user-100835,PR000263,3613.63,3545.98,NaN,微信公众号,2021-10-15 22:59:12,2021-10-15 22:59:28,否
100318,86627,sys-2021-322891956,user-116711,PR000676,802.18,748.85,NaN,微信公众号,2021-11-13 00:06:21,2021-11-13 00:06:39,否


In [23]:
#因为没有办法通过别的信息去判断渠道编号，所以选择删除
df=df.drop(df[df["渠道编号"].isnull()].index)

In [24]:
df.index=range(df.shape[0])

In [25]:
#数据清洗完成
df

,订单顺序编号,订单号,用户名,商品编号,订单金额,付款金额,渠道编号,平台类型,下单时间,付款时间,是否退款
0,8,sys-2021-306447069,user-104863,PR000499,499.41,480.42,渠道1,微信公众号,2021-01-01 01:05:50,2021-01-01 01:06:17,否
1,11,sys-2021-417411381,user-181957,PR000483,279.53,279.53,渠道1,APP,2021-01-01 01:36:17,2021-01-01 01:36:56,否
2,61,sys-2021-313655292,user-282453,PR000154,1658.95,1653.91,渠道1,微信公众号,2021-01-01 12:01:04,2021-01-01 12:03:20,否
3,78,sys-2021-311884106,user-167776,PR000215,343.25,337.12,渠道1,APP,2021-01-01 12:47:02,2021-01-01 12:47:21,否
4,81,sys-2021-375273222,user-138024,PR000515,329.04,329.04,渠道1,APP,2021-01-01 12:50:23,2021-01-01 12:50:50,否
...,...,...,...,...,...,...,...,...,...,...,...
100307,104515,sys-2020-247689274,user-254850,PR000239,251.83,179.56,渠道15,淘宝,2020-01-01 21:21:09,2021-01-01 21:21:55,否
100308,104527,sys-2020-250518601,user-291340,PR000950,1485.02,1407.60,渠道15,微信公众号,2020-01-01 21:57:45,2021-01-01 21:57:58,否
100309,104529,sys-2020-260463573,user-280181,PR000564,882.63,732.37,渠道15,APP,2020-01-01 22:05:34,2021-01-01 22:05:40,否
100310,104549,sys-2020-310196315,user-235227,PR000885,280.39,256.74,渠道15,微信公众号,2020-01-01 23:23:49,2021-01-01 23:24:03,否


In [26]:
#把清洗完成的数据保存
df.to_excel(r"D:\pandas\order2021_cleand.xlsx",index=False)